In [8]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from preprocessing import *
from sklearn.model_selection import KFold
from glob import glob

In [9]:
train_file_ds = tf.data.Dataset.list_files(['formatted_data/*/*.wav'])
batch_size = 5

In [10]:
# train_ds = train_file_ds.map(preprocess_with_resized_mel).batch(batch_size)

# for x, sampling_rate, label in train_ds.take(1):
#     print(np.max(x), np.min(x))
#     print(f'Data shape: {x.shape} - Sampling rate: {sampling_rate.numpy()} - Label: {label.numpy()}') # se non .batch c'è .decode()

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(batch_size, 192000, 2)),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
    ])

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.optimizers.Adam(learning_rate=1e-5)
metrics = [tf.metrics.SparseCategoricalAccuracy()]

In [12]:
from functools import partial

MEL_LOG_ARGS = {
    'downsampling_rate': 48000,
    'frame_length_in_s': 0.04,
    'frame_step_in_s': 0.02,
    'num_mel_bins': 40,
    'lower_frequency': 20,
    'upper_frequency': 4000
}

get_frozen_log_mel_spectrogram = partial(get_log_mel_spectrogram, **MEL_LOG_ARGS)
train_mel_ds = train_file_ds.map(get_frozen_log_mel_spectrogram)
for spectrogram, label in train_mel_ds.take(1):
    SHAPE = spectrogram.shape
SHAPE


QUA  Tensor("Shape:0", shape=(None,), dtype=int32) <unknown>
QUA  Tensor("Abs:0", dtype=float32)


TensorShape([192000, 0, 40])

In [13]:
def preprocess_with_resized_mel(filename):#, SHAPE):
    signal, label = get_frozen_log_mel_spectrogram(filename, downsampling_rate=48000, frame_length_in_s=0.04, frame_step_in_s=0.02, num_mel_bins=40, lower_frequency=20, upper_frequency=4000)
    signal.set_shape(SHAPE)
    print(tf.shape(signal), signal.shape)
    print(signal)
    mfccs = tf.signal.mfccs_from_log_mel_spectrograms(signal)[..., :]
    #mfccs = tf.expand_dims(mfccs, -1)
    mfccs = tf.image.resize(mfccs, [32, 32])
    label_id = tf.argmax(label == LABELS)

    return mfccs, label_id

In [14]:
kf = KFold(n_splits=5, random_state=7, shuffle=True)
X = glob("complete_data/*.wav")
for i, (train_index, test_index) in enumerate(kf.split(X)):

    train_files = [X[j] for j in train_index]
    test_files = [X[j] for j in test_index]

    train_file_ds = tf.data.Dataset.list_files(train_files)
    train_ds = train_file_ds.map(preprocess_with_resized_mel).batch(batch_size)
    test_file_ds = tf.data.Dataset.list_files(test_files)
    test_ds = test_file_ds.map(preprocess_with_resized_mel).batch(batch_size)

    ###
    # for example_batch, example_labels in train_ds.take(1):
    #     #print(example_batch.shape)
    #     #print(example_labels)
    #     pass

    model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(32,32,40)),#example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
    ])

    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = tf.optimizers.Adam(learning_rate=1e-5)
    metrics = [tf.metrics.SparseCategoricalAccuracy()]
    ###

    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    history = model.fit(train_ds, epochs=5, validation_data=test_ds)
    # computing statistics
    test_loss, test_accuracy = model.evaluate(test_ds)
    # training_loss = history.history['loss'][-1]
    # training_accuracy = history.history['sparse_categorical_accuracy'][-1]
    # val_loss = history.history['val_loss'][-1]
    # val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]


QUA  Tensor("Shape:0", shape=(None,), dtype=int32) <unknown>
QUA  Tensor("Abs:0", dtype=float32)
Tensor("Shape_1:0", shape=(3,), dtype=int32) (192000, 0, 40)
Tensor("Log:0", shape=(192000, 0, 40), dtype=float32)
QUA  Tensor("Shape:0", shape=(None,), dtype=int32) <unknown>
QUA  Tensor("Abs:0", dtype=float32)
Tensor("Shape_1:0", shape=(3,), dtype=int32) (192000, 0, 40)
Tensor("Log:0", shape=(192000, 0, 40), dtype=float32)
<TakeDataset element_spec=(TensorSpec(shape=(None, 32, 32, 40), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input dimension 2 must have length of at least 80 but got: 40
	 [[{{node mfccs_from_log_mel_spectrograms/dct/rfft}}]] [Op:IteratorGetNext]